In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import hvplot.pandas

In [ ]:
#data
df=pd.read_csv('Crimes_-_2001_to_Present (1).csv', low_memory=False)

In [ ]:
df

In [ ]:
#Year not complete, data may skew our findings
df = df.drop(df[df.Year == 2023].index)

In [ ]:
# Get the total number of crimes per year
total_crimes_per_year = df.groupby("Year").size()
#total_crimes_per_year

In [ ]:
#df[col].unique()
domestic_types = []
#df['all_domestic_types']
#domestic_types = [i for i in df['Primary Type'] if df['Domestic'] == True]
domestic_types = np.where(df.Domestic == True,df['Primary Type'], 'NA')
len([i for i in df['Domestic'] if i == True])

We can see that nearly 1 out of 7 cases are classified as "Domestic" in nature, but CPD's definition of Domestic crime is very broad and encompasses all crimes that are committed in or close to victim's or the perpetrator's residence. Let us create a classfication called Domestic Violence which encompasses crime types of "ASSAULT', 'BATTERY', 'CRIM SEXUAL ASSAULT', 'CRIMINAL SEXUAL ASSAULT', 'DOMESTIC VIOLENCE', 'HOMICIDE' 

In [ ]:
# Plot the total number of crimes per year
plt.plot(total_crimes_per_year.index, total_crimes_per_year.values)
plt.title("Total Number of Crimes per Year in Chicago")
plt.xlabel("Year")
plt.ylabel("Total Number of Crimes")
plt.savefig('Crime_Trend_Full_Data.png', bbox_inches='tight')
#plt.show()

# Get the total number of specific crimes per year
#specific_crimes_per_year = df[df["primary_type"] == "BURGLARY"].groupby("year").size()


In [ ]:
primary_crime_types = df["Primary Type"].unique().tolist()
len(primary_crime_types)

In [ ]:
fig, axs = plt.subplots(nrows=9, ncols=4, figsize=(25, 25))
plt.subplots_adjust(hspace=1.5)
fig.suptitle("Crime trend by Type (Historical)", fontsize=18, y=0.95)

for i, ax in zip(primary_crime_types, axs.ravel()):
    # Plot the total number of specific crimes per year
    df[df["Primary Type"] == i].groupby("Year").size().plot(ax=ax)
    #ax.plot(specific_crimes_per_year.index, specific_crimes_per_year.values)
    #plt.xticks(specific_crimes_per_year.index)
    ax.set_title("{}".format(i))
    #ax.get_legend().remove()
    ax.set_xlabel("")
plt.savefig('Crime_Trend_By_Type.png', bbox_inches='tight')
#plt.show()


In [ ]:
# Group the data by community area and year, and count the number of crimes
crimes_by_community_and_year = df.groupby(['Community Area', 'Year'])['ID'].count().reset_index()
crimes_by_community_and_year.rename(columns={'ID': 'Count'}, inplace=True)

In [ ]:
len(crimes_by_community_and_year["Community Area"].unique())

In [ ]:
import plotly.graph_objs as go
import pandas as pd
import geopandas as gpd

# Read in the community area shapefile
community_areas = gpd.read_file('Geo_Location/geo_export_c7c8d269-801a-422d-a09a-95a6aa6caed7.shp')

# Group the data by community area and year, and count the number of crimes
crimes_by_community_and_year = df.groupby(['Community Area', 'Year'])['ID'].count().reset_index()
crimes_by_community_and_year.rename(columns={'ID': 'Count'}, inplace=True)
crimes_by_community_and_year['Community Area'] = crimes_by_community_and_year['Community Area'].astype(float).astype(int).astype(str)

# Add a column with the name of the community area to the crimes_by_community_and_year dataframe
area_names = community_areas[['area_num_1', 'community']].rename(columns={'area_num_1': 'Community Area'})
crimes_by_community_and_year = pd.merge(crimes_by_community_and_year, area_names, on='Community Area')

# Merge the crime data with the community area shapefile
community_areas = community_areas.rename(columns={"area_numbe": "Community Area"})




In [ ]:
community_areas_crime = community_areas.merge(crimes_by_community_and_year, on='Community Area', how='left')

In [ ]:
# Define the choropleth map
fig = go.Figure(go.Choroplethmapbox(geojson=community_areas_crime.geometry.__geo_interface__,
                                     locations=community_areas_crime.index,
                                     z=community_areas_crime['Count'],
                                     colorscale='Reds',
                                     zmin=community_areas_crime['Count'].min(),
                                     zmax=community_areas_crime['Count'].max(),
                                     marker_opacity=0.7,
                                     marker_line_width=0.5,
                                     hovertemplate='<b>Community Area:</b> %{text}<br>' +
                                                   '<b>Crime Count:</b> %{z}<br>' +
                                                   '<extra><b>Historical Crime Trend:</b><br>' +
                                                   '{}</extra>',
                                     text=community_areas_crime['Community Area']))

# Define the hover feature
fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=9.5,
                  mapbox_center={"lat": 41.88, "lon": -87.63},
                  margin={"r":0,"t":0,"l":0,"b":0},
                  hovermode='closest')

def hover_fn(trace, points, state):
    ind = points.point_inds[0]
    ca = community_areas_crime.iloc[ind]['Community Area']
    ca_data = crimes_by_community_and_year[crimes_by_community_and_year['Community Area'] == ca]
    fig.update_traces(hovertemplate='<b>Community Area:</b> %{location}<br>' +
                                     '<b>Crime Count:</b> %{z}<br>' +
                                     '<extra><b>Historical Crime Trend:</b><br>' +
                                     "{}</extra>".format(ca_data.to_html(index=False)),text=ca_name)

fig.data[0].on_hover(hover_fn)

# Show the plot
#fig.show()